In [1]:
%cd users/keimy/git/oct2age


folder_path = "/Users/keimy/git/oct2age/data_boundary"

[Errno 2] No such file or directory: 'users/keimy/git/oct2age'
/Users/keimy/git/OCT2Age/notebook


# check date of kenshin data

In [30]:
import pandas as pd

df2016 = pd.read_csv("/Users/keimy/git/oct2age/data_kenshin/2016R2.csv", skiprows=0)
df2017 = pd.read_csv("/Users/keimy/git/oct2age/data_kenshin/2017R2.csv", skiprows=0)
df2018 = pd.read_csv("/Users/keimy/git/oct2age/data_kenshin/2018R2.csv", skiprows=0)
df_msp = pd.read_csv("/Users/keimy/git/oct2age/data_kenshin/msp.csv", skiprows=0)


# df2018から99999999を除外
df2018 = df2018[df2018.kenshindate != 99999999]
df2016.kenshindate.min(), df2018.kenshindate.max(), df_msp.date.min(), df_msp.date.max()


/var/folders/2q/37wkg3rj6hb2r8hl9r779vgw0000gn/T/ipykernel_79052/708554790.py:3: DtypeWarning: Columns (86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,106,110,111,112,113,114,115,116,117,118,119,120,121,122,123,125,126,127,130,133,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,187,188,189,190,191,192,193,194,199,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df2016 = pd.read_csv("/Users/keimy/git/oct2age/data_kenshin/2016R2.csv", skiprows=0)
/var/folders/2q/37wkg3rj6hb2r8hl9r779vgw0000gn/T/ipykernel_79052/708554790.py:4: DtypeWarning: Columns (177,178,187,188,189,190,191,192,193,194,199,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df2017 = pd.read_csv("/Users/keimy/git/oct2age/data_kenshin/2017R2.csv", skiprows=0)
/var/folders/2q/37wkg3rj6hb2r8hl9r779vgw0000gn/T/ipykernel_79052/708554790

(20160401, 20190329, '2017/10/10', '2022/7/8')

In [31]:
df_msp.head(1)

,kenshinID,sex,date,DOB,msp,time,m1,m2,m3,m4,m5
0,8871,男性,2017/3/16,1949/8/24,15,2分,3,4,6,1,1
